# Pima Indians Diabetes Regression
`sklearn` 결정 트리를 이용한 회귀
## 데이터 셋 출처
[Pima Indians Diabetes Database | Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database)  
[sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html)
### 사용 라이브러리

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score

--- 
### Data Load

In [3]:
df_pima = pd.read_csv("http://bit.ly/data-diabetes-csv")
df_pima.shape

(768, 9)

### 기본적인 학습
회귀에 대한 내용이므로, 결측치가 많았던 `Insulin`을 타겟으로 지정해 사용
#### 데이터 셋 나누기

In [4]:
label_name = "Insulin"

In [7]:
feature_names = df_pima.columns.tolist()
feature_names.remove(label_name)
feature_names

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

`Insulin`을 예측하는 모델을 만들 예정이므로, Train 세트에는 `Insulin`의 값이 존재하는 데이터만 사용할 수 있음

In [8]:
train = df_pima[df_pima["Insulin"]>0]
test = df_pima[df_pima["Insulin"]==0]
print(f"train: {train.shape}\n test: {test.shape}")

train: (394, 9)
 test: (374, 9)


In [9]:
X_train, y_train, X_test, y_test = train[feature_names], train[label_name], test[feature_names], test[label_name]
print(f"X_train: {X_train.shape}\ny_train: {y_train.shape}\nX_test: {X_test.shape}\ny_test: {y_test.shape}")

X_train: (394, 8)
y_train: (394,)
X_test: (374, 8)
y_test: (374,)
